In [1]:
# jetbot_control.py (JetBot - WebSocket Server + JetBot Control + Image Send)

import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time


# --- 로깅 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766  # 웹소켓 포트
CAMERA_WIDTH = 300     # 카메라 너비
CAMERA_HEIGHT = 300    # 카메라 높이

# --- JetBot 객체 초기화 ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리 함수 ---
async def handle_command(command: str, parameters: dict = None):
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")

    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)  # 기본 속도
            duration = parameters.get("duration", 1.0) # 기본 시간
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left")
            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0})  # 더 빠르게, 더 오래 회전
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0})
            await handle_command("forward", {"speed": 0.4, "duration": 1.0}) # 앞으로
        elif command == "stop":
            robot.stop()
        elif command == "rotate":  # 각도, 속도 파라미터
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3)  # 기본 회전 속도
                duration = abs(angle) / 45.0 *  0.7 # 각도에 비례해서 시간

                if angle > 0:
                    robot.right(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")

        elif command == "custom_command": # 사용자 지정 명령
            if parameters and "prompt" in parameters:
                 logger.info(f"Received Custom command prompt: {parameters['prompt']}")
            else:
                logger.warning("custom_command missing prompt")

        elif command == "none":
            pass  # 아무 동작도 하지 않음

        else:
            logger.warning(f"Unknown command: {command}")

    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")

# --- 웹소켓 핸들러 ---
async def websocket_handler(websocket, path):
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
                else:
                    logger.warning("Invalid message format")

                await send_image(websocket) # 이미지 보내기

            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling message: {e}")

    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")


# --- 이미지 전송 함수 ---
async def send_image(websocket):
    try:
        image = camera.value
        if image is not None:
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64})) # base64
    except Exception as e:
        logger.error(f"Error sending image: {e}")
        #raise  # 오류를 다시 발생시키지 않음


async def main():
    # 웹소켓 서버 시작
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started on port {WEBSOCKET_PORT}")
    await server.wait_closed()

# --- 스크립트 실행 (asyncio 이벤트 루프) ---
import sys

if __name__ == "__main__":
    if sys.version_info >= (3, 7):
        asyncio.run(main())  # Python 3.7 이상
    else:
        # Python 3.6 이하 (Jupyter Notebook 등)
        loop = asyncio.get_event_loop()
        try:
            loop.run_until_complete(main())
        except RuntimeError as e:
            if "Event loop is already running" not in str(e):
                raise  # 다른 RuntimeError는 다시 발생
            logger.info("Event loop already running.")  # 이미 실행 중이면 무시
        finally:
           pass # loop close 제거

DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Reseting PCA9685 MODE1 (without SLEEP) and MODE2
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFA
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFB
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFC
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFD
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x04 to register 0x01
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Setting PWM frequency to 1600 Hz
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Estimated pre-scale: 2
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Final pre-scale: 3
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2

RuntimeError: This event loop is already running

INFO:__main__:WebSocket server started on port 8766
DEBUG:websockets.protocol:server - state = CONNECTING
DEBUG:websockets.protocol:server - event = connection_made(<_SelectorSocketTransport fd=91 read=idle write=<idle, bufsize=0>>)
DEBUG:websockets.protocol:server - event = data_received(<516 bytes>)
DEBUG:websockets.server:server < GET / HTTP/1.1
DEBUG:websockets.server:server < Headers([('Host', '192.168.137.233:8766'), ('Connection', 'Upgrade'), ('Pragma', 'no-cache'), ('Cache-Control', 'no-cache'), ('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0'), ('Upgrade', 'websocket'), ('Origin', 'http://localhost:8000'), ('Sec-WebSocket-Version', '13'), ('Accept-Encoding', 'gzip, deflate'), ('Accept-Language', 'ko,en;q=0.9,en-US;q=0.8'), ('Sec-WebSocket-Key', 'lZlh7GQFbYgA6UIaE6cNIQ=='), ('Sec-WebSocket-Extensions', 'permessage-deflate; client_max_window_bits')])
DEBUG:websockets.server:server > HTT

In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time

# --- 로깅 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300

# --- JetBot 객체 초기화 ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리 함수 ---
async def handle_command(command: str, parameters: dict = None):
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")
    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left")
            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0})
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0})
            await handle_command("forward", {"speed": 0.4, "duration": 1.0})
        elif command == "stop":
            robot.stop()
        elif command == "rotate":
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3)
                duration = abs(angle) / 45.0 * 0.7
                if angle > 0:
                    robot.right(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")
        elif command == "custom_command":
            if parameters and "prompt" in parameters:
                logger.info(f"Received Custom command prompt: {parameters['prompt']}")
            else:
                logger.warning("custom_command missing prompt")
        elif command == "none":
            pass
        else:
            logger.warning(f"Unknown command: {command}")
    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")

# --- 웹소켓 핸들러 ---
async def websocket_handler(websocket, path):
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
                else:
                    logger.warning("Invalid message format")
                await send_image(websocket)
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling message: {e}")
    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")

# --- 이미지 전송 함수 ---
async def send_image(websocket):
    try:
        image = camera.value
        if image is not None:
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))
    except Exception as e:
        logger.error(f"Error sending image: {e}")

# --- 메인 함수 ---
async def main():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started on port {WEBSOCKET_PORT}")
    await server.wait_closed()

# --- 실행 함수 ---
def run():
    # 현재 실행 중인 루프가 있는지 확인하고 적절히 처리
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # Jupyter 환경에서 실행 중인 경우
        task = loop.create_task(main())
        return task
    else:
        # 일반 Python 환경
        try:
            loop.run_until_complete(main())
        except KeyboardInterrupt:
            logger.info("Shutting down...")
            robot.stop()
            camera.stop()
        finally:
            loop.close()

if __name__ == "__main__":
    try:
        run()
    except Exception as e:
        logger.error(f"Error in main execution: {e}")
    finally:
        # 리소스 정리
        robot.stop()
        camera.stop()

DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Reseting PCA9685 MODE1 (without SLEEP) and MODE2
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFA
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFB
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFC
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFD
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x04 to register 0x01
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Setting PWM frequency to 1600 Hz
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Estimated pre-scale: 2
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Final pre-scale: 3
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2